In [1]:
import pyvista
import lfricflux
import numpy
import mint
import vtk

In [2]:
inputFile = '../data/gungho/original/lfric_diag.nc'
configFile = '../configs/lfric.cfg'
# create a flux instance. The geometry is read form the netcdf Ugrid file
lf = lfricflux.LFRicFlux(configFile=configFile, inputFile=inputFile)

In [3]:
# create an empty vector field object
x, y = lf.getEdgeLonLat()
vf = lfricflux.VtkVectors(x, y, vector_field_name='velocity', cartesian=True)

In [4]:
# read the velocity data
u = mint.NcFieldRead(fileName=inputFile, varName='u_in_w2h').data()
v = mint.NcFieldRead(fileName=inputFile, varName='v_in_w2h').data()

In [5]:
# last dimension is number of edges
u.shape

(1, 30, 3072)

In [6]:
# set the vector field
vf.setField(u, v)
# set time = 0, elev = 0
vf.setSlice((0, 0))

In [12]:
# vector field
arrow = vtk.vtkArrowSource()
glyphs = vtk.vtkGlyph3D()
glyphs.SetScaleModeToScaleByVector()
glyphs.SetColorModeToColorByVector()
glyphs.SetScaleFactor(1./50.) # 50 m/s -> 1 radius
glyphs.SetRange(0., 50.)
glyphs.SetInputData(vf.getMesh())
glyphs.SetSourceConnection(arrow.GetOutputPort())

In [14]:
pyvista.global_theme.cmap = 'jet'
p = pyvista.Plotter(window_size=(900, 800))
p.add_mesh(glyphs)
p.show()

Widget(value="<iframe src='http://localhost:64637/index.html?ui=P_0x181374430_3&reconnect=auto' style='width: …

In [15]:
from ipywidgets import interact
import ipywidgets as widgets

def callbackElev(k):
    # copy the new values
    vf.setSlice((0, k))
    print(f'k = {k}')
    vf.vectorField.Modified()

interact(callbackElev, k=widgets.IntSlider(min=0, max=u.shape[1] - 1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='k', max=29), Output()), _dom_classes=('widget-interact',…

<function __main__.callbackElev(k)>